In [48]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stat

city        = 'london'
outfolder   = '../ProcessedData/' + city + '/'

In [3]:
venues_users = {}
for ind, line in enumerate(open(outfolder + '/venues_info/london_venues_users.dat')):
    #if ind == 100: break
    fields = line.strip().split('\t')
    venue = fields[0]
    users = fields[1:] 
    venues_users[venue] = users

In [5]:
users_venues = {}

for ind, line in enumerate(open(outfolder + 'user_info/london_user_venues_full_locals_filtered.dat')):
    #if ind == 100: break
    fields = line.strip().split('\t')
    user   = fields[0]
    venues = [fff.split(',')[0] for fff in fields[1:]]
    users_venues[user] = venues
    
print len(users_venues)

101771


In [10]:
print len(users_venues)
print len(venues_users)

101771
1074517


In [35]:
city          = 'london'
outfolder     = '../ProcessedData/' + city + '/'
successdata   = outfolder + 'venues_info/' + city + '_venues_success_measures.csv'
successdata_L = outfolder + 'venues_info/' + city + '_venues_success_measures_local_like.csv'

#successmeasures     = {}
for line in open(successdata):
    if 'checkinsCount' not in line:
        fields = line.strip().split('\t')
        successmeasures[fields[0]] = [float(fff) for fff in fields[1:]]

for line in open(successdata_L):
    if 'Llike' not in line:
        fields = line.strip().split('\t')
        successmeasures[fields[0]].append(float(fields[1]))
        
for k, v in successmeasures.items():
    if len(v) == 3:
        successmeasures[k].append(0.0)

In [36]:
for ind, (k, v) in enumerate(successmeasures.items()):
    if ind == 10: break
        
    print ind, k, v

0 4fb2d653e4b0886effe1aa86 [1.0, 6.0, 1.0, 0.0]
1 51e72fb78bbd22a2ac38a4d5 [0.0, 3.0, 3.0, 1.44316673930647]
2 4dde1ba7227106a58b839c56 [0.0, 40.0, 13.0, 0.0]
3 4cbadba60180721e6d5e9c61 [1.0, 418.0, 59.0, 0.8824184397531647]
4 4cf4079088de3704cb80822b [1.0, 44.0, 11.0, 0.0]
5 4beec861e24d20a121587314 [1.0, 45.0, 37.0, 0.8696102805667949]
6 52e77b7e498e93f4116873b7 [0.0, 23.0, 2.0, 0.0]
7 4fb14b50e4b0c546fb6cd55e [0.0, 12.0, 10.0, 0.0]
8 4ac518daf964a520c9a820e3 [26.0, 2357.0, 1942.0, 25.01860489138895]
9 4bbb8c3f51b89c743a5a862a [2.0, 618.0, 105.0, 1.0602742878345426]


In [28]:
len(successmeasures)

101178

In [68]:
users_successmeasures = {}

for ind, (user, venues) in enumerate(users_venues.items()):
    #if ind == 10: break
    for v in venues:
        if v in successmeasures:            
            if user not in users_successmeasures:
                users_successmeasures[user] = {}
                users_successmeasures[user]['tipCount']      = [successmeasures[v][0]]
                users_successmeasures[user]['checkinsCount'] = [successmeasures[v][1]]
                users_successmeasures[user]['usersCount']    = [successmeasures[v][2]]
                users_successmeasures[user]['lLondonLike']   = [successmeasures[v][3]]
            else:
                users_successmeasures[user]['tipCount'].append(successmeasures[v][0])
                users_successmeasures[user]['checkinsCount'].append(successmeasures[v][1])
                users_successmeasures[user]['usersCount'].append(successmeasures[v][2])
                users_successmeasures[user]['lLondonLike'].append(successmeasures[v][3])

In [69]:
users_success_stats = {}

for user, measures in users_successmeasures.items():
    
    users_success_stats[user] = {}
    
    for measure, values in measures.items():   
        users_success_stats[user][measure + '_avg']     = np.mean(values)
        users_success_stats[user][measure + '_std']     = np.std(values)
        users_success_stats[user][measure + '_entropy'] = stat.entropy(np.asarray(values), base = len(values))

     

In [71]:

df = pd.DataFrame.from_dict(users_success_stats, orient = 'index') 
df = df.fillna(0)
df.to_csv(outfolder + '/venues_info/USERS_likes_successmeas.dat', na_rep='nan')
df.head()

,tipCount_avg,tipCount_std,lLondonLike_std,usersCount_avg,lLondonLike_entropy,checkinsCount_avg,usersCount_std,usersCount_entropy,checkinsCount_entropy,checkinsCount_std,tipCount_entropy,lLondonLike_avg
100001187,185.833333,202.383231,254.951143,22614.083333,0.768071,28607.750000,25429.369252,0.741642,0.763361,30362.940515,0.756986,245.626948
100002121,62.000000,0.000000,0.000000,2429.000000,0.000000,4883.000000,0.000000,0.000000,0.000000,0.000000,0.000000,55.962459
10000216,350.000000,366.432349,195.147069,37367.666667,0.662945,52601.333333,35383.966278,0.613079,0.771035,38791.882917,0.497830,245.713340
10000221,26.000000,0.000000,0.000000,1435.000000,0.000000,2896.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.786776
100006753,30.000000,0.000000,0.000000,1788.000000,0.000000,2403.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.498092


In [72]:
len(df), len(df.keys())

(98634, 12)